In [1]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np


In [2]:
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

In [3]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[3, 5, 5, 5], [3, 6, 6, 5], [3, 6, 6, 5]]

In [4]:

file = open('43.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]

fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]

In [5]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [18, 18, 18, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [18, 18, 18, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [17, 17, 17, 17, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [17, 17, 17, 17, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1],
 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 7, 5, 5, 5, 1],
 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 7, 5, 5, 5, 1],
 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 5, 5, 5, 1],
 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 5, 5, 5, 1],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1]]

In [6]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        
        v_y = model.cbGetSolution(model._y)
        #print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')

        print(model.cbGet(GRB.Callback.MIPSOL_OBJBND))
        
        bsp = gp.Model("Subproblem") 
        #variables
        x = {}
        for i in range(locations):
            for j in range(customers):
                x[i,j] = bsp.addVar(obj=varcost[i][j])
        
        #demand constraint
        demand_constr = {} 
        for j in range(customers):
            demand_constr[j] = bsp.addConstr(sum(x[i,j] for i in range(locations)) >= demand[j])
        
        #capacity constraint
        cap_constr = {}
        for i in range(locations):
            cap_constr[i] = bsp.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
        
        #logical constraint
        log_constr = {}
        for i in range(locations):
            for j in range(customers):
                log_constr[i,j] = bsp.addConstr(x[i,j] <= v_y[i,j] * bigM[i][j])
        
        bsp.optimize()
        
        
        print('\n\n',bsp.Status,'\n\n')
        
        if bsp.Status == 2:
         #feasible subproblem
            #update upper bound
            fc = sum(fixcost[i][j]*v_y[i,j] for i in range(locations) for j in range(customers))
            model.addConstr(model._z <= fc + bsp.objVal)
            
            v = np.zeros(customers) #dual of demand constraints
            for j in range(customers):
                v[j] = demand_constr[j].pi #get dual value

            w = np.zeros((locations,customers)) #dual of logical constraints
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = log_constr[i,j].pi

            u = np.zeros(locations) #dual of capacity constraints
            for i in range(locations):
                u[i] = cap_constr[i].pi

            cm = np.zeros((locations,customers)) #coefficient of y in master problem
            for i in range(locations):
                for j in range(customers):
                    cm[i,j] = fixcost[i][j] + bigM[i][j] * w[i,j]

                
            model.cbLazy( model._z >= sum(demand[j] * v[j] for j in range(customers)) + 
                         sum(capacity[i] * u[i] for i in range(locations)) + 
                         sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers))) 
            
#            print(model._z >= sum(demand[j] * v[j] for j in range(customers)) + sum(capacity[i] * u[i] for i in range(locations)) + 
#                     sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))

#                print('\n\n')
#                print(sum(demand[j] * v[j] for j in range(customers)))
#                print(sum(capacity[i] * u[i] for i in range(locations)))
#                print([w[i,j] for i in range(locations) for j in range(customers)])
#                print(bsp.objVal)

                
                # save result of x from subproblem
            global transport
            for i in range(locations):
                for j in range(customers):
                    transport[i,j] = x[i,j].x
            print(transport)

#            print(cm)
#            print(v)
#            print(w)
#            print(u)

        else:
            infbsp = gp.Model('Infeasible subproblem')
            dummy = infbsp.addVar(obj = 1)
            
            v = {}
            for j in range(customers): v[j] = infbsp.addVar(lb=0)
            w = {}
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = infbsp.addVar(lb=0)
            u = {}
            for i in range(locations): u[i] = infbsp.addVar(lb=0)
            
            infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) -  
                              sum(capacity[i]*u[i] for i in range(locations)) -  
                              sum(bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) == 0.0001 )                
            for i in range(locations):
                for j in range(customers):
                    infbsp.addConstr( v[j] - u[i] - w[i,j] <= 0);
            infbsp.addConstr(dummy == 0)
            
            #print(infbsp.Status)

            infbsp.modelSense = -1 #set to maximization
            infbsp.optimize()
            #print(u)
            #print(w)            
            #positive dual variables => use - instead of +
            model.cbLazy( sum(demand[j] * v[j].x for j in range(customers)) -
                sum(capacity[i] * u[i].x for i in range(locations)) -  
                sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
            
#            print(sum(demand[j] * v[j].x for j in range(customers)) - 
#                sum(capacity[i] * u[i].x for i in range(locations)) - 
#                sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
            
#            print('\n\n')
#            print(sum(demand[j] * v[j].x for j in range(customers)))
#            print(sum(capacity[i] * u[i].x for i in range(locations)))
#            print([w[i,j].x for i in range(locations) for j in range(customers)])
#            infbsp.dispose()
                    
                   
        bsp.dispose()
                       

In [ ]:

# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

for j in range(customers):
    m.addConstr(sum(y[i,j] for i in range(locations)) >= 1)
        
m.Params.lazyConstraints = 1
m._y = y
m._z = z

transport = np.zeros((locations, customers)) # to save result of x from subproblem
m.optimize(subproblem)
print(m.objBound)

print("Objective: " + str(m.objVal))
for i in range(locations):
    print([y[i,j].x for j in range(customers)])

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = transport[i,j]
        
print(solution)

Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 15 rows, 226 columns and 225 nonzeros
Model fingerprint: 0x36fa8909
Variable types: 1 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Current LB:  0.0 

-1e+100
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xdf51283b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 225 rows and 0 columns
Presolve time: 0.00s
Presolved: 30 rows, 225 columns, 450 nonzeros

Iteration    Objecti

Presolve removed 225 rows and 208 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 723 nonzeros
Model fingerprint: 0xe6baf310
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 227 rows and 256 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x735bfe9e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01


Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x3e0160ee
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 242 rows and 206 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 732 nonzeros
Model fingerprint: 0x0d5a520f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 207 rows and 222 columns
Presolve time: 0.01s
Presolved: 20 rows, 34 columns, 90 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf

Model fingerprint: 0xc41e8a02
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 192 rows and 203 columns
Presolve time: 0.00s
Presolved: 35 rows, 53 columns, 153 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.000000e-05   0.000000e+00      0s
      29   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 29 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  0.0 

0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xf840e053
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 237 rows and 191 columns
Presolve time: 0.00s
Presolved: 18 rows, 34 columns, 67 nonze

Optimize a model with 227 rows, 256 columns and 730 nonzeros
Model fingerprint: 0x00ba1bc6
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 227 rows and 256 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  9776.0 

8780.940789473683
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xa1f239a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 225 rows and 201 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 s

...

       0   -0.0000000e+00   1.052632e-05   0.000000e+00      0s
      16   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13235.000000000002 

12744.901254020599
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x08572e5a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 247 rows and 214 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 730 nonzeros
Model fingerprint: 0x4a199361
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
P

Model fingerprint: 0x76bd724d
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 213 rows and 231 columns
Presolve time: 0.01s
Presolved: 14 rows, 25 columns, 59 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.111111e-05   0.000000e+00      0s
      11   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13149.000000000005 

12757.020224664579
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x656f6ea1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-15, 2e+01]
Presolve removed 247 rows and 215 columns
Presolve time: 0.03s

Solved in 

  RHS range        [1e+00, 2e+01]
Presolve removed 244 rows and 211 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 729 nonzeros
Model fingerprint: 0x01659461
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 210 rows and 227 columns
Presolve time: 0.01s
Presolved: 17 rows, 29 columns, 69 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.111111e-05   0.000000e+00      0s
      18   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
 63322 27128 13025.6977   65   24 13257.0000 12773.3803  3.65%  12.6  105s
Current LB:  13036.999999999998 

12773.451397049323
Gurobi Optimizer version 9

Optimal objective -0.000000000e+00
Current LB:  13170.999999999998 

12782.503528792186
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0x97be7368
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-16, 2e+01]
Presolve removed 241 rows and 208 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 227 rows, 256 columns and 730 nonzeros
Model fingerprint: 0x3f4fb02c
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 1e-04]
Presolve removed 208 rows and 223 columns
Presolve time: 0.01s
Presolved: 19 rows, 33 columns

  RHS range        [1e-04, 1e-04]
Presolve removed 210 rows and 224 columns
Presolve time: 0.01s
Presolved: 17 rows, 32 columns, 75 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.052632e-05   0.000000e+00      0s
      14   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Current LB:  13184.0 

12789.209876899427
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 225 columns and 675 nonzeros
Model fingerprint: 0xc4c8841a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-16, 2e+01]
Presolve removed 247 rows and 214 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


 3 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.2 bui

In [ ]:
help()

In [ ]:
help(GRB.attr)

In [ ]:
m.objBound

In [ ]:
y0

In [ ]:
k = 1
mvars = m.getVars()
y0 = np.zeros((locations, customers))
for i in range(locations):
    for j in range(customers):
        y0[i,j] = int(round(mvars[k].x,0))
        k+=1

In [ ]:
y0

In [ ]:
model0 = gp.Model()
x0 = {}
for i in range(locations):
    for j in range(customers):
        x0[i,j] = model0.addVar(obj=varcost[i][j])

#demand constraint
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) >= demand[j])
        
#capacity constraint
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

#logical constraint
log_constr = {}
for i in range(locations):
    for j in range(customers):
        log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i][j] * min(capacity[i], demand[j]))

model0.optimize()

print("Objective: " + str(model0.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x0[i,j].x

print(solution)

In [ ]:
89 + sum(y0[i][j]*fixcost[i][j] for i in range(locations) for j in range(customers))

In [ ]:
also allowing not satisfying customer + unsatisfying demand variable added with very high punish cost => when returning to master problem always 

In [ ]:
help(GRB.attr)

In [ ]:


            
        
        # save result of x from subproblem
        global transport
        for i in range(locations):
            for j in range(customers):
                transport[i,j] = x[i,j].x
        print(transport)
        print(bsp.objVal)

In [ ]:
http://web.stanford.edu/class/msande348/papers/bendersingams
https://www.imus.us.es/IMUS-MSRI2016/assets/media/docs/lectures/lec_blanco_1.pdf